In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
# warnings.simplefilter('ignore', UserWarning)

import numpy as np
import pandas as pd
import pycircstat as circ
from scipy import stats

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_kinematic as fk

In [47]:
def calculate(ds_label, shifts_ds):

    residuals = np.linalg.norm(shifts_ds[['pref_1', 'pref_2']].to_numpy() - shifts_ds[['pref_1', 'pref_1']].to_numpy(), axis=1)
    signed_residuals = shifts_ds['pref_2'].values - shifts_ds['pref_1'].values


    # This is a circular measure, so wrap the residuals
    if 'pd' in ds_label:
        bound = 180

        # Normal residuals
        wrapped_residuals = fk.wrap(residuals, bound=bound)
        wrapping_idxs = np.argwhere(residuals > bound).flatten()
        if len(wrapping_idxs) > 0:
            residuals[wrapping_idxs] = bound - wrapped_residuals[wrapping_idxs]

        # signed residuals
        wrapped_signed_residuals = fk.wrap_negative(signed_residuals, bound=bound)
        wrapping_idxs = np.argwhere(signed_residuals > bound).flatten()
        # if len(wrapping_idxs) > 0:
        #     signed_residuals[wrapping_idxs] = bound - wrapped_signed_residuals[wrapping_idxs]

            
    elif 'po' in ds_label:
        bound = 90
        
        # Normal residuals
        wrapped_residuals = fk.wrap(residuals, bound=bound)
        wrapping_idxs = np.argwhere(residuals > bound).flatten()
        if len(wrapping_idxs) > 0:
            residuals[wrapping_idxs] = bound - wrapped_residuals[wrapping_idxs]

        # signed residuals
        wrapped_signed_residuals = fk.wrap_negative(signed_residuals, bound=bound)
        wrapping_idxs = np.argwhere(signed_residuals > bound).flatten()
        # if len(wrapping_idxs) > 0:
        #     signed_residuals[wrapping_idxs] = bound - wrapped_signed_residuals[wrapping_idxs]
    else:
        pass

    

    # Calculate the mean and std of the residuals
    mean_residual = np.mean(residuals)
    std_residual = np.std(residuals)
    print(f"Residual mean: {mean_residual}, Residual std: {std_residual}")

    # Calculate mean difference
    mean_signed_residuals = np.mean(signed_residuals)
    print(f"Mean Signed Residual: {mean_signed_residuals}")

    # Calculate the RMSE of the residuals
    rmse_residual = np.sqrt(np.mean(residuals ** 2))
    print(f"RMSE: {rmse_residual}")


    # Calculate the Pearson R and pvalue of the shifts
    pearson_r, pval = stats.pearsonr(shifts_ds['pref_2'], shifts_ds['pref_1'])
    print(f"Pearson's R: {pearson_r}, p: {pval}")

    return residuals, rmse_residual, mean_residual, std_residual, pearson_r, pval


def calculate_circular(ds_label, shifts_ds):

    pref1 = np.deg2rad(shifts_ds['pref_1'].values)
    pref2 = np.deg2rad(shifts_ds['pref_2'].values)

    circ_mean_diff = circ.mean(circ.pairwise_cdiff(pref2, pref1))
    circ_mean_diff = np.rad2deg(circ_mean_diff)
    print(f"Circular Mean Difference: {circ_mean_diff}")

    circ_corr = circ.corrcc(pref1, pref2)
    t_score = circ_corr * np.sqrt(len(pref1) - 2) / np.sqrt(1 - circ_corr ** 2)
    dist = stats.beta(len(pref1)/2 - 1, len(pref1)/2 - 1, loc=-1, scale=2)
    p = 2*dist.cdf(-abs(circ_corr))
    print(f"Circular Correlation: {circ_corr}, p: {p}")

    # kuiper_test = circ.kuiper(pref1, pref2, res=500)
    # print(f"Kuiper Test p: {kuiper_test[0]}, kuiper test statistic: {kuiper_test[1]}")

    # cm_test = circ.cmtest(pref1, pref2)
    # print(f"CM Test p: {cm_test[0]}, cm test statistic: {cm_test[1]}")

    watson_test = circ.watson_williams(pref1, pref2)
    print(f"Watson Williams Test p: {watson_test[0]}")


In [50]:
type = 'repeat'  # 'repeat', 'multi'
lighting = 'normal'  # 'normal', 'dark'
rig = 'VWheelWF'  # 'VWheelWF', 'VTuningWF', 'ALL'
match_type = 'curated'  # 'curated', 'all'

data_path = os.path.join(r"H:\thesis\WF_Figures\full", f'{type}_{lighting}_{rig}')
print(data_path)

if 'ALL' in data_path:
    settings_dict = {'session_shorthand': ['free', 'fixed'],
                     'sort': 'rig',
                     'vis_resp_type': 'both'}
elif 'VWheelWF' in data_path:
    settings_dict = {'session_shorthand': ['session1', 'session2'],
                     'sort': 'slug',
                     'vis_resp_type': 'both'}
elif 'VTuningWF' in data_path:
    settings_dict = {'session_shorthand': ['session1', 'session2'],
                     'sort': 'slug',
                     'vis_resp_type': 'all'}
else:
    raise ValueError('data_path not recognized')

ds_list = [f'osi_shifts_vis_resp_{match_type}_matches_{settings_dict["vis_resp_type"]}_vis_resp',
           f'dsi_shifts_vis_resp_{match_type}_matches_{settings_dict["vis_resp_type"]}_vis_resp',
           f'po_shifts_vis_resp_{match_type}_matches_{settings_dict["vis_resp_type"]}_vis_resp',
           f'pd_shifts_vis_resp_{match_type}_matches_{settings_dict["vis_resp_type"]}_vis_resp']

for ds_label in ds_list:
    print(ds_label)

    with pd.HDFStore(os.path.join(data_path, 'stats.hdf5'), 'r') as f:
        shifts = f[ds_label][:]

    residuals, rmse_residual, mean_residual, std_residual, pearson_r, pval = calculate(ds_label, shifts)

    # Calculate the circular correlation of the residuals (only valid for PO and PD)
    if 'po_' in ds_label or 'pd_' in ds_label:
        calculate_circular(ds_label, shifts)

        # # Do it on a per-mouse basis
        # all_mice = shifts.groupby('mouse').count().index
        # mice_with_enough_cells = all_mice[shifts.groupby('mouse').day.count() > 3]

        # for mouse in mice_with_enough_cells:
        #     print(mouse)
        #     mouse_shifts = shifts[shifts['mouse'] == mouse]
        #     print(f"Number of matches: {len(mouse_shifts)}")

        #     mouse_residuals, mouse_rmse_residual, mouse_mean_residual, mouse_std_residual, mouse_pearson_r, mouse_pval = calculate(ds_label, mouse_shifts)
        #     calculate_circular(mouse_shifts)
        #     print('\n')

    print('\n')

H:\thesis\WF_Figures\full\repeat_normal_VWheelWF
osi_shifts_vis_resp_curated_matches_both_vis_resp
Residual mean: 0.1637060299433885, Residual std: 0.17328432341587258
Mean Signed Residual: 0.10660967567467962
RMSE: 0.2383843975211514
Pearson's R: 0.5101345351808639, p: 0.0017439094975474235


dsi_shifts_vis_resp_curated_matches_both_vis_resp
Residual mean: 0.27261241244708767, Residual std: 0.15630471293609316
Mean Signed Residual: 0.11842311033340452
RMSE: 0.31424304400615705
Pearson's R: 0.4938520325815782, p: 0.0025696980676579374


po_shifts_vis_resp_curated_matches_both_vis_resp
Residual mean: 14.77927283137704, Residual std: 17.27388974334769
Mean Signed Residual: 8.698539937016893
RMSE: 22.7335472878654
Pearson's R: 0.678372614764204, p: 7.522350425499216e-06
Circular Mean Difference: 8.738515215344771
Circular Correlation: 0.683249661204028, p: 6.093636424624296e-06
Watson Williams Test p: 0.46374241793337234


pd_shifts_vis_resp_curated_matches_both_vis_resp
Residual mean: 68

c:\Users\mmccann\miniconda3\envs\prey_capture\lib\site-packages\pycircstat-0.0.2-py3.8.egg\pycircstat\tests.py:415: UserWarning: Test not applicable. Average resultant vector length < 0.45.
  warnings.warn(
